We need to compare an unequally spaced version of something we know with something equally spaced.

In [1]:
using Multitaper, Plots
using LombScargle, Random

include(dirname(dirname(pathof(Multitaper)))*"/Examples/pgram.jl")
include(dirname(dirname(pathof(Multitaper)))*"/Examples/single_tapers.jl")
include(dirname(dirname(pathof(Multitaper)))*"/Examples/matern.jl")

function fejer(N)
  fey = [(1/N)*(1-cos(N*x))/(1-cos(x)) for x in LinRange(-1.0, 1.0, N+1)[1:N]]
  fey[Int64(N/2 + 1)] = N
  return fftshift(N*fey/sum(fey))[vcat(N, collect(1:N-1))]
end 


LoadError: [91mcould not open file /Users/charlottehaley/.julia/packages/Multitaper/MeP12/Examples/matern.jl[39m

In [2]:
N = 1_024 
N2 = 513
freq = LinRange(-0.5, 0.5, N+1)[1:N]

phi = 1.0
a = 0.1
nu = 1.5
flr = 0.0

S = maternsdf.(freq, phi, a, nu, flr);

LoadError: [91mUndefVarError: maternsdf not defined[39m

In [3]:
dat = Matern_sim([phi, a, nu, flr], N)[1];

LoadError: [91mUndefVarError: Matern_sim not defined[39m

In [4]:
Pxx = sm_pgram(dat, taper = :dirichlet)
Direct = sm_pgram(dat)

LoadError: [91mUndefVarError: dat not defined[39m

In [5]:
plot(Pxx)
plot!(Direct)
plot!(freq, S*4, xlims = [0, 0.5], label = "Truth")

LoadError: [91mUndefVarError: Pxx not defined[39m

In [6]:
MT = multispec(dat)

LoadError: [91mUndefVarError: dat not defined[39m

In [7]:
plot!(MT, label = "MT, NW = 4", linewidth = 2)

LoadError: [91mUndefVarError: MT not defined[39m

## Unequal spacing

This is not actually unequal spacing. It is an equal grid and I have deleted some observations. 

In [8]:
# I will delete exactly 24 points, so the results should be GOOD
times = unique(sort(vcat(1, randperm(N)[1:1000], N)))
plot(times, dat[times])
scatter!(times, dat[times])

LoadError: [91mUndefVarError: dat not defined[39m

In [9]:
w = 4.0/N
bet = 0.5

Sb = bspec(Float64.(times), dat[times], w, 3, bet, 0.0, false) # (times, dat, W, K, beta, nz, Ftest)

LoadError: [91mUndefVarError: dat not defined[39m

In [10]:
pl = plot(Sb, label = "Bronez Multitaper")
plot!(pl, freq, S, xlims = [0, 0.5], label = "Truth")

LoadError: [91mUndefVarError: Sb not defined[39m

In [11]:
LS = lombscargle(Float64.(times), dat[times], normalization=:psd)

LoadError: [91mUndefVarError: dat not defined[39m

In [12]:
f,P = freqpower(LS)
length(f)

LoadError: [91mUndefVarError: LS not defined[39m

In [13]:
# Note the odd division by 4 to compare magnitudes
plot!(pl, f[1:2500],P[1:2500]/4, yscale = :log10, label = "Lomb-Scargle", linealpha = 0.1, c = :black)

LoadError: [91mUndefVarError: f not defined[39m

In [14]:
SW = lombscargle(Float64.(times), ones(length(times)), center_data = false, 
    fit_mean = false, normalization=:psd)
f,PW = freqpower(SW)

spw = plot(f,PW, yscale=:log10, label = "Lomb-Scargle Spectral Window", xlims = [0,0.5])

FK = fejer(N)
plot!(spw, LinRange(0, 1, N+1)[1:N], FK, yscale = :log10, label="Féjèr")

LoadError: [91mUndefVarError: fejer not defined[39m

## Missing data problem

In [16]:
Smd = mdmultispec(Float64.(times), dat[times]);

LoadError: [91mUndefVarError: dat not defined[39m

In [17]:
plot(Sb, label = "Bronez spectrum")
plot!(Smd.f, Smd.S/2, label="Missing data MT")
plot!(freq, S, xlims = [0, 0.5], label = "Truth")

LoadError: [91mUndefVarError: Sb not defined[39m

Disappointing performance on all counts.